# FHIR Implementation Guide Testing Pipeline
This notebook provides a comprehensive pipeline for automatically extracting requirements from FHIR Implementation Guides and generating executable test suites. The pipeline transforms Implementation Guide (IG) documentation into structured test code that can validate FHIR server implementations.

## Overview
This automated pipeline takes FHIR Implementation Guide documentation and produces comprehensive test suites through several integrated stages:

- Implementation Guide Preparation: Convert and clean IG HTML documentation to markdown format
- Requirements Extraction: Use AI to identify and extract testable requirements from the IG
- Requirements Refinement: Consolidate and refine the extracted requirements
- Requirements Downselection: Combine multiple requirement sets and remove duplicates
- Test Plan Generation: Convert requirements into detailed test specifications
- Test Kit Generation: Generate executable Inferno test code

## Running this Notebook
The notebook is structured to run each stage sequentially. You can either:

- Run the complete pipeline: Execute all cells to process a complete IG
- Run individual stages: Execute specific sections as needed

Each stage will prompt you for inputs or use sensible defaults. The pipeline automatically saves intermediate outputs in checkpoint directories for review and iteration.

### Pipeline Stages
#### Stage 1: Implementation Guide Preparation
Text Extraction and Cleaning
- Converts HTML IG files to markdown format using Markdownify
- Cleans unnecessary content (navigation, headers, formatting artifacts)
- Prepares clean, structured text for AI processing
Inputs: HTML files from FHIR IG downloads
Outputs: Clean markdown files in checkpoints/text_extraction/ and checkpoints/post_processing/

#### Stage 2: Requirements Extraction
AI-Powered Requirements Identification
- Processes markdown files using AI to extract clear, testable requirements
- Formats requirements according to INCOSE Systems Engineering standards
- Generates structured requirements with IDs, descriptions, actors, and conformance levels
- Handles large documents through intelligent chunking
- Provides proper attribution and source tracking
Inputs: Cleaned markdown files
Outputs: Structured requirements list in checkpoints/requirements_extraction/ in markdown file

#### Stage 3: Requirements Refinement
AI-Based Requirements Consolidation
- Filters and identifies only testable requirements from raw extractions
- Consolidates duplicate requirements and merges related ones
- Applies consistent formatting and structure
- Removes non-testable assertions and narrative content
Inputs: Raw requirements from extraction stage in markdown file
Outputs: Refined requirements list in checkpoints/revised_reqs_extraction/

#### Stage 4: Requirements Downselection
Multi-Source Requirements Integration
- Combines multiple requirements lists from different extraction runs
- Uses semantic similarity analysis to identify and remove duplicates
- Creates a comprehensive, deduplicated final requirements set
- Provides statistical analysis of the consolidation process
Inputs: Multiple refined requirements files in markdown or JSON format
Outputs: Final consolidated requirements in checkpoints/requirements_downselect/ in markdown file

#### Stage 5: Test Plan Generation
Comprehensive Test Specification Development
- Transforms requirements into detailed test specifications
- Analyzes IG capability statements for context
- Generates implementation strategies with specific FHIR operations
- Creates structured test plans with validation criteria
Inputs: Refined requirements and IG capability statements in markdown format
Outputs: Detailed test plan in checkpoints/testplan_generation/ in markdown file

#### Stage 6: Test Kit Generation
Executable Test Code Creation
- Converts test specifications into executable Inferno Ruby tests
- Generates complete test suites with proper file organization
- Creates modular test structures following Inferno framework patterns
- Includes validation and alignment checking
Inputs: Test plan specification in markdown format
Outputs: Complete Inferno test kit in checkpoints/testkit_generation/

### Output Structure
The pipeline generates organized outputs in checkpoint directories:
checkpoints/
├── text_extraction/          # Converted markdown files
├── post_processing/          # Cleaned markdown files  
├── requirements_extraction/   # Initial AI-extracted requirements
├── revised_reqs_extraction/  # Refined requirements lists
├── requirements_downselect/  # Final consolidated requirements
├── testplan_generation/     # Detailed test specifications
└── testkit_generation/      # Executable Inferno test suites
Each stage preserves its outputs, allowing for iteration, review, and alternative processing paths.

## Setup

#### Importing Notebooks as Modules (from the [Jupyter Notebook Documentation](https://jupyter-notebook.readthedocs.io/en/4.x/examples/Notebook/rstversions/Importing%20Notebooks.html))

In [1]:
import inspect
import json
import llm_utils
import importlib
import tiktoken
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from glob import glob

## Initializing LLM Clients

In [2]:
importlib.reload(llm_utils)
llm_clients = llm_utils.LLMApiClient()

In [3]:
llm_clients.clients

{'claude': <anthropic.Anthropic at 0x1192674a0>,
 'gemini': genai.GenerativeModel(
     model_name='models/gemini-2.5-pro',
     generation_config={'max_output_tokens': 8192, 'temperature': 0.3},
     safety_settings={<HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>},
     tools=None,
     system_instruction=None,
     cached_content=None
 ),
 'gpt': <openai.OpenAI at 0x1192ee330>}

## Text Extraction

### HTML to Markdown Conversion

In [5]:
import html_narrative_extractor #import html extractor module

# Process directory with default settings
result = html_narrative_extractor.convert_local_html_to_markdown(
    "../us-core/site", #input directory
    "checkpoints/markdown1/" #output directory
)

Found 717 HTML files to process
Processed 10/717 files
Processed 20/717 files
Processed 30/717 files
Processed 40/717 files
Processed 50/717 files
Processed 60/717 files
Processed 70/717 files
Processed 80/717 files
Processed 90/717 files
Processed 100/717 files
Processed 110/717 files
Processed 120/717 files
Processed 130/717 files
Processed 140/717 files
Processed 150/717 files
Processed 160/717 files
Processed 170/717 files
Processed 180/717 files
Processed 190/717 files
Processed 200/717 files
Processed 210/717 files
Processed 220/717 files
Processed 230/717 files
Processed 240/717 files
Processed 250/717 files
Processed 260/717 files
Processed 270/717 files
Processed 280/717 files
Processed 290/717 files
Processed 300/717 files
Processed 310/717 files
Processed 320/717 files
Processed 330/717 files
Processed 340/717 files
Processed 350/717 files
Processed 360/717 files
Processed 370/717 files
Processed 380/717 files
Processed 390/717 files
Processed 400/717 files
Processed 410/717

### Markdown Post-processing

In [6]:
import markdown_cleaner #import markdown cleaner module
markdown_cleaner.process_directory("checkpoints/markdown1", #input directory
                                   "checkpoints/markdown2/") #output directory

Found 717 markdown files in checkpoints/markdown1
Cleaned and saved: checkpoints/markdown2/StructureDefinition-us-core-smokingstatus-testing.md
Cleaned and saved: checkpoints/markdown2/DocumentReference-episode-summary.md
Cleaned and saved: checkpoints/markdown2/ValueSet-us-core-diagnosticreport-report-and-note-codes.md
Cleaned and saved: checkpoints/markdown2/StructureDefinition-us-core-head-circumference-mappings.md
Cleaned and saved: checkpoints/markdown2/Observation-10-minute-apgar-respiratory-effort.md
Cleaned and saved: checkpoints/markdown2/StructureDefinition-us-core-tribal-affiliation-definitions.md
Cleaned and saved: checkpoints/markdown2/DiagnosticReport-metabolic-panel.md
Cleaned and saved: checkpoints/markdown2/Observation-serum-potassium.md
Cleaned and saved: checkpoints/markdown2/StructureDefinition-us-core-genderIdentity-definitions.md
Cleaned and saved: checkpoints/markdown2/StructureDefinition-us-core-heart-rate-testing.md
Cleaned and saved: checkpoints/markdown2/Sear

## Requirements Extraction

### Prompt-based Requirement Extraction

In [11]:
import reqs_extraction #import LLM requirements extraction module

In [ ]:
reqs_extraction.run_requirements_extractor(
    'checkpoints/markdown2', #input directory
    'checkpoints/requirements_extraction/us-core', #output directory
    'claude', #LLM API of choice
    llm_clients)

INFO:root:Found markdown directory at /Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/markdown2
INFO:root:Found 793 markdown files
INFO:root:Processing with claude...
INFO:root:Starting processing with claude on directory: /Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/markdown2
INFO:root:Found 793 markdown files:
INFO:root:  - StructureDefinition-us-core-smokingstatus-testing.md
INFO:root:  - DocumentReference-episode-summary.md
INFO:root:  - ValueSet-us-core-diagnosticreport-report-and-note-codes.md
INFO:root:  - StructureDefinition-us-core-head-circumference-mappings.md
INFO:root:  - Observation-10-minute-apgar-respiratory-effort.md
INFO:root:  - StructureDefinition-us-core-tribal-affiliation-definitions.md
INFO:root:  - DiagnosticReport-metabolic-panel.md
INFO:root:  - Observation-serum-potassium.md
INFO:root:  - StructureDefinition-us-core-genderIdentity-definitions.md
INFO:root:  - StructureDefinition-us-core-heart-rate-testing.md
INFO:root:  - Se


Processing Implementation Guide with Claude...
This may take several minutes depending on the size of the Implementation Guide.


INFO:root:  - SearchParameter-us-core-procedure-patient.md
INFO:root:  - SearchParameter-us-core-organization-name-testing.md
INFO:root:  - Specimen-specimen-example-urine.md
INFO:root:  - SearchParameter-us-core-condition-encounter.md
INFO:root:  - Organization-acme-lab.md
INFO:root:  - StructureDefinition-us-core-authentication-time-definitions.md
INFO:root:  - SearchParameter-us-core-practitioner-name.md
INFO:root:  - CodeSystem-condition-category-testing.md
INFO:root:  - SearchParameter-us-core-encounter-patient.md
INFO:root:  - MedicationRequest-self-tylenol.md
INFO:root:  - Condition-condition-duodenal-ulcer.md
INFO:root:  - Observation-PHQ9-item-example-69722-7.md
INFO:root:  - Procedure-defib-implant.md
INFO:root:  - StructureDefinition-us-core-average-blood-pressure-definitions.md
INFO:root:  - StructureDefinition-us-core-procedure-testing.md
INFO:root:  - ValueSet-us-core-screening-assessment-observation-category.md
INFO:root:  - changes.md
INFO:root:  - SearchParameter-us-co


Processing complete!
Generated requirements document: checkpoints/requirements_extraction/us-core/claude_reqs_list_v1_20250825_105114.md
Processed 793 files


## Requirement Refinement with LLM

In [13]:
import reqs_reviewer
importlib.reload(reqs_reviewer)

<module 'reqs_reviewer' from '/Users/ceadams/Documents/onclaive/onclaive/pipeline/reqs_reviewer.py'>

### Large Number of Requirements (500+)

In [ ]:
result = reqs_reviewer.run_batch_requirements_refinement(
    input_file="checkpoints/requirements_extraction/us-core/claude_reqs_list_v1_20250825_105114.md",
    llm_client_instance=llm_clients,
    batch_size=50,
    api_type="claude"
)

INFO:root:Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts


STARTING BATCH PROCESSING
Input: checkpoints/requirements_extraction/us-core/claude_reqs_list_v1_20250825_105114.md
Output: checkpoints/revised_reqs_extraction
Batch size: 50 requirements
API: claude

File size: 1,453,237 characters
Splitting requirements...
Found 2241 total requirements
Will process in 45 batches

BATCH 1/45
   Requirements: 50 (#1-#50)
   Size: 48,033 chars (~12,008 tokens)


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


   Completed in 8.0s
   Pausing 2s...
   Progress: 1/45 (2.2%)
   ETA: 7.4 minutes remaining

BATCH 2/45
   Requirements: 50 (#51-#100)
   Size: 34,740 chars (~8,685 tokens)


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


   Completed in 105.7s
   Pausing 2s...
   Progress: 43/45 (95.6%)
   ETA: 3.5 minutes remaining

BATCH 44/45
   Requirements: 50 (#2151-#2200)
   Size: 33,984 chars (~8,496 tokens)
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 minutes...
      Request taking longer than 8 m

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


   Completed in 75.6s
   Progress: 45/45 (100.0%)
   ETA: 0.0 minutes remaining

COMBINING RESULTS
--------------------
Merging batch results and renumbering...
   Processing batch 1 results...
   Processing batch 2 results...
   Processing batch 3 results...
   Processing batch 4 results...
   Processing batch 5 results...
   Processing batch 6 results...
   Processing batch 7 results...
   Processing batch 8 results...
   Processing batch 9 results...
   Processing batch 10 results...
   Processing batch 11 results...
   Processing batch 12 results...
   Processing batch 13 results...
   Processing batch 14 results...
   Processing batch 15 results...
   Processing batch 16 results...
   Processing batch 17 results...
   Processing batch 18 results...
   Processing batch 19 results...
   Processing batch 20 results...
   Processing batch 21 results...
   Processing batch 22 results...
   Processing batch 23 results...
   Processing batch 24 results...
   Processing batch 25 results..

INFO:anthropic._base_client:Retrying request to /v1/messages in 0.484711 seconds
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


### Small Number of Requirements (<500)

In [7]:
# Manual refinement with specific file
refinement_result = reqs_reviewer.refine_requirements(
    input_file="/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/requirements_extraction/us-core/gpt_reqs_list_v1_20250820_091156.md",
    api_type="claude",
    output_dir="checkpoints/revised_reqs_extraction",
    llm_client_instance=llm_clients
)

print(f"✅ Refined requirements saved to: {refinement_result['output_file']}")
print(f"📊 {refinement_result['original_requirements_count']} → {refinement_result['requirements_count']} requirements")

if refinement_result.get('warnings'):
    print("⚠️  Warnings:")
    for warning in refinement_result['warnings']:
        print(f"  - {warning}")

2025-08-24 13:15:19,212 - root - INFO - Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts
2025-08-24 13:15:19,213 - root - INFO - Starting requirements refinement with claude
2025-08-24 13:15:19,233 - root - INFO - Original requirements count: 3595
2025-08-24 13:15:19,233 - root - INFO - Input size: 1455002 characters, ~363750 tokens
2025-08-24 13:15:19,234 - root - INFO - Sending requirements to claude for refinement...
2025-08-24 13:15:19,236 - root - INFO - Estimated input tokens: 364328
2025-08-24 13:15:19,236 - root - INFO - API input limit: 180000
2025-08-24 13:15:19,236 - root - INFO - API output limit: 8192
2025-08-24 13:15:19,236 - root - WARNING - Input may exceed token limits. Splitting into chunks...
2025-08-24 13:15:19,238 - root - INFO - Prompt overhead: 578 tokens
2025-08-24 13:15:19,238 - root - INFO - Available tokens for requirements: 177422
2025-08-24 13:15:19,257 - root - INFO - Found 3596 individual requirements to process
2025-08-24 

KeyboardInterrupt: 

## Requirement Downselection

In [41]:
import requirement_downselect
importlib.reload(requirement_downselect)
requirement_downselect.full_pass(
    md_files=["/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_125844.md", "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_125935.md", "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_130022.md", "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_130106.md", "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_130905.md", "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_133736.md", "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_135555.md" ,"/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250805_144850.md"]
    #rag_files=["checkpoints/requirements_extraction/RAG/raw_output.json"]
    )

/Users/ceadams/Documents/onclaive/onclaive/pipeline/requirement_downselect.py:23: SyntaxWarning: invalid escape sequence '\*'
  headings = list(re.finditer("\*\*\w+\*\*\:", split))
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-mpnet-base-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Final number of requirements: 30
Output saved in Markdown format in directory: checkpoints/requirements_downselect


## Test Plan Generation

In [8]:
import logging
llm_clients.logger.setLevel(logging.INFO)

##### Without RAG- Faster

In [9]:
import req_to_testplan
importlib.reload(req_to_testplan)

result = req_to_testplan.generate_consolidated_test_plan(
    llm_clients,
    'claude',  # 'claude' or 'gemini' or 'gpt'
    llm_clients.logger,
    "/Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250730_122703.md",
    "/Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md",
    "Plan-Net IG",
    output_dir='checkpoints/testplan_generation'
)

print(f"Generated test plan with improved capability parsing: {result['test_plan_path']}")

INFO:llm_utils:Starting test plan generation with claude for Plan-Net IG
INFO:llm_utils:Parsed 19 requirements from /Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250730_122703.md
INFO:llm_utils:Parsed capability statement from /Users/ceadams/Documents/onclaive/onclaive/full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md
INFO:llm_utils:Identifying group for requirement REQ-01 using claude...
INFO:llm_utils:Identifying group for requirement REQ-02 using claude...
INFO:llm_utils:Identifying group for requirement REQ-03 using claude...
INFO:llm_utils:Identifying group for requirement REQ-04 using claude...
INFO:llm_utils:Identifying group for requirement REQ-05 using claude...
INFO:llm_utils:Identifying group for requirement REQ-06 using claude...
INFO:llm_utils:Identifying group for requirement REQ-07 using claude...
INFO:llm_utils:Identifying group for requirement REQ-08 using claude...
INFO:llm_utils:Identifying group

Generated test plan with improved capability parsing: checkpoints/testplan_generation/claude_test_plan_20250730_123104.md


##### With RAG

In [ ]:
import warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set logging level to reduce noise
import logging
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
logging.getLogger("backoff").setLevel(logging.ERROR)

import req_to_testplan_rag
importlib.reload(req_to_testplan_rag)

#clearing any existing capability statements from vector database
req_to_testplan_rag.clear_capability_collection("capability_statements")

Deleted existing collection: capability_statements


ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))


In [ ]:
req_to_testplan_rag.generate_consolidated_test_plan(
    llm_clients, 
    'claude', 
    llm_clients.logger, 
    "checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250730_122703.md", 
    "checkpoints/markdown2/CapabilityStatement-us-core-server.md", 
    "US Core IG",
    output_dir='checkpoints/testplan_generation',
    verbose=True)

INFO:llm_utils:Starting test plan generation with claude for Plan-Net IG
INFO:llm_utils:Parsed 19 requirements from /Users/ceadams/Documents/onclaive/onclaive/pipeline/checkpoints/revised_reqs_extraction/claude_reqs_list_v2_20250730_122703.md
INFO:llm_utils:Initialized capability collection from ../full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md
INFO:llm_utils:Identifying group for requirement REQ-01 using claude...
INFO:llm_utils:Identifying group for requirement REQ-02 using claude...
ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))
INFO:llm_utils:Identifying group for requirement REQ-03 using claude...
INFO:llm_utils:Identifying group for requirement REQ-04 using claude...
INFO:


Processing Group: General/Cross-Resource (15 requirements)

Processing REQ-03: Health Plan API Must Support data population capability

RAG RETRIEVAL FOR REQ-03
Query: Health Plan API Must Support data population capability "Health Plan API actors SHALL be capable of populating all Must Support data elements as part of the query results." Health Plan API FHIR SHALL
Searching for 5 most relevant capability chunks...

Found 5 matching chunks:

  Match 1 (distance: 0.7221184968948364):
  Length: 3036 chars
  Preview: FHIR Resource Detail: #### HealthcareService

Conformance Expectation: **SHALL**

Supported Profiles...
  ...lthcareService?_lastUpdated=[_lastUpdated]` |

---

  Match 2 (distance: 0.7407464981079102):
  Length: 3288 chars
  Preview: FHIR Resource/Component: ### RESTful Capabilities by Resource/Profile:

**Summary of Search Criteria...
  ...nerRole:network, PractitionerRole:endpoint |  |  |

  Match 3 (distance: 0.8931890726089478):
  Length: 3225 chars
  Preview: FHIR Reso

ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)'))))


Completed test specification for REQ-03


INFO:llm_utils:Processing requirement for group 'General/Cross-Resource': REQ-04
INFO:llm_utils:Generating test specification for REQ-04 using claude...


...

INFO:llm_utils:Processing requirement for group 'Security': REQ-18
INFO:llm_utils:Generating test specification for REQ-18 using claude...


Completed test specification for REQ-02

Processing Group: Security (1 requirements)

Processing REQ-18: Plan-Net Server unauthorized request rejection

RAG RETRIEVAL FOR REQ-18
Query: Plan-Net Server unauthorized request rejection "A server SHALL reject any unauthorized requests by returning an `HTTP 401` unauthorized response code." Plan-Net Server FHIR SHALL
Searching for 5 most relevant capability chunks...

Found 5 matching chunks:

  Match 1 (distance: 0.5878045558929443):
  Length: 1169 chars
  Preview: FHIR Resource/Component: ### FHIR RESTful Capabilities

The Plan\-Net Server **SHALL**:

1. Support ...
  ...eturning an `HTTP 401` unauthorized response code.

  Match 2 (distance: 0.7881736159324646):
  Length: 996 chars
  Preview: FHIR Major Section: ## CapabilityStatement: Plan\-Net CapabilityStatement

| *Official URL*: http://...
  ...openapi.json) \| [Download](plan-net.openapi.json)

  Match 3 (distance: 0.9208030700683594):
  Length: 802 chars
  Preview: FHIR Major Secti

INFO:llm_utils:Consolidated test plan saved to checkpoints/testplan_generation/claude_test_plan_20250730_140520.md


Completed test specification for REQ-18


{'requirements_count': 19,
 'group_count': 4,
 'test_plan_path': 'checkpoints/testplan_generation/claude_test_plan_20250730_140520.md'}

## Test Kit Generation

In [ ]:
import plan_to_tests
import importlib
importlib.reload(plan_to_tests)

plan_to_tests.generate_inferno_test_kit(
    llm_clients,
    'gpt',
    'checkpoints/testplan_generation/claude_test_plan_20250730_140520.md',
    #'../test_kit_dev/inferno-guidance.md',
    output_dir='checkpoints/testkit_generation',
    expected_actors=["Server", "Client"]
)

/Users/ceadams/Documents/onclaive/onclaive/pipeline/plan_to_tests.py:775: SyntaxWarning: invalid escape sequence '\A'
  """
INFO:plan_to_tests:Starting Inferno test generation with gpt for PlanNet
INFO:plan_to_tests:Parsed test plan into 4 sections
INFO:plan_to_tests:Found 19 total requirements
INFO:plan_to_tests:Loaded Inferno DSL guidance
INFO:plan_to_tests:Processing section: General/Cross-Resource with 15 requirements
INFO:plan_to_tests:Generating tests for section: General/Cross-Resource
INFO:plan_to_tests:Sending 122 tokens to gpt API (limit: 16000)
INFO:plan_to_tests:Attempting to generate tests for entire section: General/Cross-Resource


Found 19 potential requirements
Processing requirement: REQ-03
Added requirement REQ-03 to section General/Cross-Resource
Processing requirement: REQ-04
Added requirement REQ-04 to section General/Cross-Resource
Processing requirement: REQ-05
Added requirement REQ-05 to section General/Cross-Resource
Processing requirement: REQ-06
Added requirement REQ-06 to section General/Cross-Resource
Processing requirement: REQ-07
Added requirement REQ-07 to section General/Cross-Resource
Processing requirement: REQ-08
Added requirement REQ-08 to section General/Cross-Resource
Processing requirement: REQ-09
Added requirement REQ-09 to section General/Cross-Resource
Processing requirement: REQ-10
Added requirement REQ-10 to section General/Cross-Resource
Processing requirement: REQ-11
Added requirement REQ-11 to section General/Cross-Resource
Processing requirement: REQ-12
Added requirement REQ-12 to section General/Cross-Resource
Processing requirement: REQ-13
Added requirement REQ-13 to section G

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:plan_to_tests:Generating test for requirement: REQ-03
INFO:plan_to_tests:Requirement REQ-03: Sending 1111 tokens to gpt API (limit: 16000)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:plan_to_tests:Successfully generated test for requirement: REQ-03
INFO:plan_to_tests:Generating test for requirement: REQ-04
INFO:plan_to_tests:Requirement REQ-04: Sending 1135 tokens to gpt API (limit: 16000)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:plan_to_tests:Successfully generated test for requirement: REQ-04
INFO:plan_to_tests:Generating test for requirement: REQ-05
INFO:plan_to_tests:Requirement REQ-05: Sending 1140 tokens to gpt API (limit: 16000)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:plan_to_tests:Successfully generated test for requirement

{'total_sections': 4,
 'total_requirements': 19,
 'generated_tests': 19,
 'module_dir': 'checkpoints/testkit_generation/gpt_testkit_20250819_225027/plannet',
 'module_file': 'checkpoints/testkit_generation/gpt_testkit_20250819_225027/gpt_plannet_20250819_225027.rb',
 'output_dir': 'checkpoints/testkit_generation/gpt_testkit_20250819_225027',
 'timestamp': '20250819_225027'}

In [28]:
bool(True and None)

False